In [251]:
rung_num = int((input("Number of Rungs:")))
bc_n = rung_num
gamma = 1.76e7  # Gyromagnetic ratio in rad/s/T
B = float(input("Enter the magnetic field strength (in Tesla): "))     # input for magnetic field B in Tesla

# Calculate the frequency in Hz
frequency_hz = gamma * B

# Convert frequency to MHz
frequency_mhz = f"{frequency_hz} / 1e6 Mhz"



In [252]:
from pyaedt import Hfss

# Launch HFSS Desktop in GUI mode
hfss = Hfss(
    non_graphical=False,  
    new_desktop=True,     
    close_on_exit=False   
)
hfss.modeler.model_units = "cm"  # Set model units
hfss.solution_type = "Terminal"
print("HFSS Initialized")


PyAEDT INFO: Python version 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.17.5.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\hafis\AppData\Local\Temp\pyaedt_hafis_4342d923-5b02-47dd-83ff-2c6de39e3fba.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT WARNING: Only AEDT Student Version found on the system. Using Student Version.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 57535.
PyAEDT INFO: Electronics Desktop started on gRPC port: 57535 after 11.206382036209106 seconds.
PyAEDT INFO: AEDT installation Path D:\Ansys Student\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 Student version started with process ID 34952.
PyAEDT WARNING: AEDT started as Student version, but requested as Regular version.


In [253]:
# define base design variables
hfss["bc_rung_t"] = "2cm"    # Thickness of rung
hfss["bc_rung_l"] = "65cm"   # Full length of rung
hfss["bc_port_h"] = "1cm"    # Port height
hfss["bc_port_w"] = "2cm"    # Port width
hfss["bc_r"] = "32.5cm"      # Coil radius
hfss["bc_depth"] = "1.4mil"  # Sheet thickness
hfss["bc_n"] = bc_n           # Number of rungs

print("Base variables added.")


Base variables added.


In [254]:
# define dependent parametric variables
hfss["bc_theta"] = "360deg/bc_n"
hfss["bc_angle_port"] = "acos((bc_port_h^2 - 2*bc_r^2)/(-2*bc_r^2))"
hfss["bc_angle_rung"] = "bc_theta - bc_angle_port"
hfss["bc_r_inner"] = "bc_r*cos(bc_angle_rung/2)"
hfss["bc_rung_w"] = "2*bc_r_inner*tan(bc_angle_rung/2)"

print("Dependent variables added.")


Dependent variables added.


In [255]:
# create open range based on operation frequency 
hfss.create_open_region(frequency=f"{frequency_mhz}", boundary='Radiation', apply_infinite_ground=False, gp_axis='-z')

# create the first rectangle: vertical rung part
rung_rect = hfss.modeler.create_rectangle(
    origin=["-bc_rung_t/2", 0, "bc_port_h/2"],
    sizes=["(bc_rung_l/2 - bc_port_h/2)", "bc_rung_t"],
    orientation="XZ",
    name="BcRungTop",
    material="copper"
)
print("BcRungTop created.")

# create the second rectangle: horizontal bus bar at top
bus_rect = hfss.modeler.create_rectangle(
    origin=["-bc_rung_w/2", 0, "bc_rung_l/2"],
    sizes=["-bc_rung_t", "bc_rung_w"],
    orientation="XZ",
    name="BcBusTop",
    material="copper"
)
print("BcBusTop created.")

# unite the two rectangles and capture the new object that is created
t_structure = hfss.modeler.unite([rung_rect, bus_rect])
print("✅ T-structure created successfully! Check in AEDT.")

# assign Perfect E boundary to the new united object before duplicating
print("Assigning Perfect E boundary to T-structure...")
hfss.assign_perfect_e(t_structure, "PE_T_Structure")
print("✅ Perfect E boundary assigned.")


PyAEDT INFO: Open Region correctly created.
PyAEDT INFO: Project Project39 Saved correctly
PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec
BcRungTop created.
BcBusTop created.
PyAEDT INFO: Union of 2 objects has been executed.
✅ T-structure created successfully! Check in AEDT.
Assigning Perfect E boundary to T-structure...
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Boundary Perfect E PerfectE_BcRungTop has been created.
✅ Perfect E boundary assigned.


In [256]:
hfss.modeler.duplicate_and_mirror(
    rung_rect,                 # assignment
    origin=[0, 0, 0],          # mirror base point
    vector=[0, 0, 1],          # normal vector along Z 
    duplicate_assignment=True  
)


['BcRungTop_1']

In [257]:

duplicates = ["BcRungTop", "BcRungTop_1"]  
hfss.modeler.move(duplicates, ["0mm", "bc_r_inner", "0mm"])


True

In [258]:
bc_n_value = int(float(hfss["bc_n"].split()[0]))  # extract 8 from bc_n

# compute angle per step
angle_per_clone = 360 / bc_n_value


In [259]:
status, new_objects = hfss.modeler.duplicate_around_axis(
    assignment=["BcRungTop", "BcRungTop_1"],  # original pair
    axis="Z",                                # Rotate about Z-axis
    angle=angle_per_clone,                   # Numeric angle per step
    clones=bc_n_value, 
    create_new_objects=True,
    duplicate_assignment=True
)

print(f"✅ Created {bc_n_value} positions for rungs around Z-axis.")


✅ Created 8 positions for rungs around Z-axis.


In [260]:
# 1. Create Polyline
polyline_points = [
    ["bc_rung_w/2", "bc_r_inner", "bc_rung_l/2"],
    ["bc_r*sin(bc_angle_rung/2+bc_angle_port)",
     "bc_r*cos(bc_angle_rung/2+bc_angle_port)",
     "bc_rung_l/2"]
]
cap_polyline = hfss.modeler.create_polyline(points=polyline_points, cover_surface=False, name="CapTopPath")
print("✅ Polyline for BcCapTop created.")

# 2. Sweep
cap_top = hfss.modeler.sweep_along_vector(
    assignment=cap_polyline,
    sweep_vector=["0mm", "0mm", "-bc_port_w"],
    draft_angle=0,
    draft_type="Round"
)
cap_top.name = "BcCapTop"
print("✅ Sweep complete. Created BcCapTop.")

# ------------------------------------------------------------------
# Add Lumped Port to BcCapTop before duplicating
# ------------------------------------------------------------------
print("Adding lumped port to BcCapTop...")
# Define the 3D start and stop points for the integration line
line_start = [polyline_points[0][0], polyline_points[0][1], f"({polyline_points[0][2]}) - bc_port_w/2"]
line_stop = [polyline_points[1][0], polyline_points[1][1], f"({polyline_points[1][2]}) - bc_port_w/2"]

# CORRECTED: Use the 'integration_line' argument with a list of two points
hfss.lumped_port(
    assignment=cap_top.name,
    integration_line=[line_start, line_stop],
    name="CapPort"  # The 'name' argument is valid according to the docstring
)
print("✅ Lumped port 'CapPort' created on BcCapTop.")
# ------------------------------------------------------------------


# 3. Mirror to create BcCapBottom
before_objs = set(hfss.modeler.object_names)
hfss.modeler.duplicate_and_mirror(
    assignment=cap_top,
    origin=[0, 0, 0],
    vector=[0, 0, -1],  # Normal vector
    duplicate_assignment=True
)
after_objs = set(hfss.modeler.object_names)
new_obj_name = list(after_objs - before_objs)[0]
hfss.modeler[new_obj_name].name = "BcCapBottom"
print(f"✅ Created BcCapBottom: {new_obj_name}")

# 4. Duplicate both around Z-axis
bc_n_value = int(float(hfss["bc_n"].split()[0]))
angle_per_clone = 360 / bc_n_value
hfss.modeler.duplicate_around_axis(
    assignment=["BcCapTop", "BcCapBottom"],
    axis="Z",
    angle=angle_per_clone,
    clones=bc_n_value,
    create_new_objects=True,
    duplicate_assignment=True
)
print(f"✅ Duplicated both caps {bc_n_value} times around Z-axis.")

✅ Polyline for BcCapTop created.
✅ Sweep complete. Created BcCapTop.
Adding lumped port to BcCapTop...
PyAEDT INFO: Boundary AutoIdentify CapPort has been created.
PyAEDT ERROR: **************************************************************
PyAEDT ERROR:   File "<frozen runpy>", line 198, in _run_module_as_main
PyAEDT ERROR:   File "<frozen runpy>", line 88, in _run_code
PyAEDT ERROR:   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 608, in run_forever
PyAEDT ERROR:     self._run_once()
PyAEDT ERROR:   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 1936, in _run_once
PyAEDT ERROR:     handle._run()
PyAEDT ERROR:   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\events.py", line 84, in _run
PyAEDT ERROR:     self._context.run(self._callback, *s

In [261]:
# -----------------------------
# 1. Create BcPort Rectangle
# -----------------------------
port_rect = hfss.modeler.create_rectangle(
    origin=["-bc_port_w/2", "bc_r_inner", "-bc_port_h/2"],  # Parametric origin
    sizes=["bc_port_h", "bc_port_w" ],  # Width and height
    orientation="XZ",                  # Axis = Y → XZ plane
    name="BcPort",
    material="copper"
)
print("✅ Created BcPort rectangle.")

# ------------------------------------------------------------------
# Add Lumped Port to BcPort before duplicating
# ------------------------------------------------------------------
print("Adding lumped port to BcPort...")
# Define the start and stop points for the vertical integration line
line_start = ["0cm", "bc_r_inner", "-bc_port_h/2"] # Bottom-center
line_stop = ["0cm", "bc_r_inner", "bc_port_h/2"]  # Top-center

# CORRECTED: Use the 'integration_line' argument with a list of two points
hfss.lumped_port(
    assignment=port_rect,
    integration_line=[line_start, line_stop],
    name="RungPort",
)
print("✅ Lumped port 'RungPort' created on BcPort.")
# ------------------------------------------------------------------


# -----------------------------
# 2. Duplicate Around Z-axis
# -----------------------------
# Use parametric variable bc_theta for angle
hfss.modeler.duplicate_around_axis(
    assignment=port_rect,
    axis="Z",
    angle="bc_theta",             # Use parametric angle (360deg/bc_n)
    clones=hfss["bc_n"],          # Number of duplicates
    create_new_objects=True,
    duplicate_assignment=True
)
print("✅ BcPort duplicated around Z-axis using bc_theta.")

✅ Created BcPort rectangle.
Adding lumped port to BcPort...
PyAEDT INFO: Boundary AutoIdentify RungPort has been created.
PyAEDT ERROR: **************************************************************
PyAEDT ERROR:   File "<frozen runpy>", line 198, in _run_module_as_main
PyAEDT ERROR:   File "<frozen runpy>", line 88, in _run_code
PyAEDT ERROR:   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 608, in run_forever
PyAEDT ERROR:     self._run_once()
PyAEDT ERROR:   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 1936, in _run_once
PyAEDT ERROR:     handle._run()
PyAEDT ERROR:   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\events.py", line 84, in _run
PyAEDT ERROR:     self._context.run(self._callback, *self._args)
PyAEDT ERROR:   File "C:\Users\